In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 50.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=27968ae76d075bbc57f3971e78fd122d775db7efadf6e2420b0b4473288ea2ec
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
sc = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
sc

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DataPath= "/content/drive/MyDrive/access.log"

In [7]:
# weblogs = sc.read.csv(DataPath)
# weblogs= sc.read.options(Map("delimiter"->",")).csv(DataPath)
# weblogs = sc.read.csv(DataPath, sep=' ',inferSchema=True, header=False)

In [8]:
# weblogstext = sc.read.text(DataPath)

In [9]:
# import pandas as pd

# df = pd.read_csv('/content/drive/MyDrive/access.log', sep=" ")
# df

In [10]:
# weblogstext.take(1)

In [11]:
rdd1 = sc.sparkContext.textFile(DataPath)

In [41]:
rdd1.take(3)

['54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"']

In [13]:
blanks = rdd1.map(lambda x:','.join(x or '0' for x in x.split(' ')))
blanks.take(1)

['54.36.149.41,-,-,[22/Jan/2019:03:56:14,+0330],"GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53,HTTP/1.1",200,30577,"-","Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)","-"']

In [14]:
# Split the row into list
import csv
from io import StringIO
def split(line):
    reader = csv.reader(StringIO(line))
    return next(reader)

weblogsrdd=blanks.map(split)
weblogsrdd.take(10)

[['54.36.149.41',
  '-',
  '-',
  '[22/Jan/2019:03:56:14',
  '+0330]',
  'GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53,HTTP/1.1',
  '200',
  '30577',
  '-',
  'Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)',
  '-'],
 ['31.56.96.51',
  '-',
  '-',
  '[22/Jan/2019:03:56:16',
  '+0330]',
  'GET,/image/60844/productModel/200x200,HTTP/1.1',
  '200',
  '5667',
  'https://www.zanbil.ir/m/filter/b113',
  'Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mobile,Safari/537.36',
  '-'],
 ['31.56.96.51',
  '-',
  '-',
  '[22/Jan/2019:03:56:16',
  '+0330]',
  'GET,/image/61474/productModel/200x200,HTTP/1.1',
  '200',
  '5379',
  'https://www.zanbil.ir/m/filter/b113',
  'Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3

In [15]:
from pyspark.sql import Row

In [16]:
weblogsrdd.take(1)

[['54.36.149.41',
  '-',
  '-',
  '[22/Jan/2019:03:56:14',
  '+0330]',
  'GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53,HTTP/1.1',
  '200',
  '30577',
  '-',
  'Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)',
  '-']]

In [17]:
import numpy as np 

In [18]:
dataframe = weblogsrdd.map(lambda x:Row(client=x[0],datetime=(x[3] +','+ x[4]),method=x[5].split(','),rescode=x[6],size=x[7],user_agent=x[9]))

In [19]:
dataframe.take(5)

[Row(client='54.36.149.41', datetime='[22/Jan/2019:03:56:14,+0330]', method=['GET', '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', '27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', 'p53', 'HTTP/1.1'], rescode='200', size='30577', user_agent='Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)'),
 Row(client='31.56.96.51', datetime='[22/Jan/2019:03:56:16,+0330]', method=['GET', '/image/60844/productModel/200x200', 'HTTP/1.1'], rescode='200', size='5667', user_agent='Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mobile,Safari/537.36'),
 Row(client='31.56.96.51', datetime='[22/Jan/2019:03:56:16,+0330]', method=['GET', '/image/61474/productModel/200x200', 'HTTP/1.1'], rescode='200', size='5379', user_agent='Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mo

In [45]:
df12 = dataframe.filter(lambda x: len(x.method) >= 3)

In [46]:
df12.count()

10365096

In [47]:
dataframe1 = df12.map(lambda x:Row(client=x[0],datetime=x[1],rescode=x[3],method=x[2][0], request = x[2][1],protocol = x.method[-1],size=x[4] ,user_agent=x[5]) )

[Row(client='80.82.64.127', datetime='[22/Jan/2019:08:40:11,+0330]', method=['\\x16\\x03\\x01\\x00\\x96\\x01\\x00\\x00\\x92\\x03\\x03\\xCB\\x077\\xE6\\x83Nq>\\xB5\\xED\\x9E\\x87\\xB7\\x08\\xD7\\xEAF'], rescode='400', size='166', user_agent='-'),
 Row(client='72.52.125.78', datetime='[22/Jan/2019:09:55:32,+0330]', method=['\\x17\\x10\\x05\\x15JL\\xAD\\xDC\\xCEL\\xC1Ff>F'], rescode='400', size='166', user_agent='-'),
 Row(client='108.178.16.154', datetime='[22/Jan/2019:14:32:00,+0330]', method=['j\\x00\\xA7\\x00\\xF3\\x00H\\x00z\\x00\\xF3\\x00\\xFE\\x00\\xFB\\x00\\x8E\\x00\\xFE\\x00H\\x00\\x8E\\x00H\\x00\\xF3\\x00\\xA7\\x00\\xD9\\x00\\xA7\\x00\\xF3\\x00\\xB0\\x00H\\x00\\x8B\\x00\\xF3\\x00\\xC5\\x00\\xC5\\x00\\xFC\\x00\\x8E\\x00\\xFB\\x00\\xFC\\x00\\xEC\\x00\\x8B\\x00\\xF3\\x00\\xD9\\x00\\xA7\\x00\\xFB\\x00\\xFC\\x00\\xEC\\x00\\x8E\\x00\\xF3\\x00\\xFB\\x00\\xA7\\x00\\xC5\\x00\\xF3\\x00\\xEC\\x00\\x09\\x00\\x17\\x00\\xD9\\x00\\xFB\\x00z\\x00\\xD9\\x00\\xC5\\x00\\xC5\\x00z\\x00\\x17\\x00\\x

In [ ]:
#after  request
dataframe1.count()

In [30]:
dataframe1.count()

10365152

In [49]:
dataframe1.take(10)

[Row(client='54.36.149.41', datetime='[22/Jan/2019:03:56:14,+0330]', rescode='200', method='GET', request='/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', protocol='HTTP/1.1', size='30577', user_agent='Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)'),
 Row(client='31.56.96.51', datetime='[22/Jan/2019:03:56:16,+0330]', rescode='200', method='GET', request='/image/60844/productModel/200x200', protocol='HTTP/1.1', size='5667', user_agent='Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mobile,Safari/537.36'),
 Row(client='31.56.96.51', datetime='[22/Jan/2019:03:56:16,+0330]', rescode='200', method='GET', request='/image/61474/productModel/200x200', protocol='HTTP/1.1', size='5379', user_agent='Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mobile,Safari/537.36'),
 Row(client='40.77.167.129', datetime='[2

In [22]:
dataframe.count()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [50]:
dataframe2 = dataframe1.map(lambda x:Row(host=x[0],datetime=x[1],rescode=x[2],size=x[3],method=x[4],request=x[5],protocol=x[6] 
                                        ,user_agent = 'Bot' if 'AhrefsBot' in x[7] else 'Mobile' if 'Mobile' in x[7] else "Windows" if 'Windows' in x[7] else "Bot" if "bingbot" in x[7] else "Bot" if 'Googlebot' in x[7] else "Applebot" if 'Applebot' in x[7] else 'Unknown' ) )

In [52]:
dataframe2.count()

10365096

In [ ]:
# user_agents = dataframe1.map(lambda x: x.user_agent)

In [ ]:
# user_agents.take(100)

In [ ]:
# devices = user_agents.map(lambda x:Row(device = 'Mobile' if 'Mobile' in x else "Windows" if 'Windows' in x else "Bingbot" if "bingbot" in x else "Googlebot" if 'Googlebot' in x else "Applebot" if 'Applebot' in x else  "Unknown" ))

In [ ]:
# devices.take(200)

In [ ]:
import urllib.parse
encodedStr = '%D9%85%D8%A7%DB%8C%DA%A9%D8%B1%D9%88%D9%81%D8%B1-%D8%B1%D9%88%D9%85%DB%8C%D8%B2%DB%8C-%D8%B3%D8%A7%D9%85%D8%B3%D9%88%D9%86%DA%AF-%D9%85%D8%AF%D9%84-CE288'
urllib.parse.unquote_plus(encodedStr)

In [ ]:
dataframe2

In [53]:
from pyspark.sql import SQLContext, Row

In [54]:
sqlC=SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [55]:
WebLogsFile=dataframe2.toDF()

In [56]:
WebLogsFile.show()

+-------------+--------------------+-------+----+--------------------+--------+--------+----------+
|         host|            datetime|rescode|size|              method| request|protocol|user_agent|
+-------------+--------------------+-------+----+--------------------+--------+--------+----------+
| 54.36.149.41|[22/Jan/2019:03:5...|    200| GET|/filter/27|13%20%...|HTTP/1.1|   30577|       Bot|
|  31.56.96.51|[22/Jan/2019:03:5...|    200| GET|/image/60844/prod...|HTTP/1.1|    5667|    Mobile|
|  31.56.96.51|[22/Jan/2019:03:5...|    200| GET|/image/61474/prod...|HTTP/1.1|    5379|    Mobile|
|40.77.167.129|[22/Jan/2019:03:5...|    200| GET|/image/14925/prod...|HTTP/1.1|    1696|       Bot|
|  91.99.72.15|[22/Jan/2019:03:5...|    200| GET|/product/31893/62...|HTTP/1.1|   41483|   Windows|
|40.77.167.129|[22/Jan/2019:03:5...|    200| GET|/image/23488/prod...|HTTP/1.1|    2654|       Bot|
|40.77.167.129|[22/Jan/2019:03:5...|    200| GET|/image/45437/prod...|HTTP/1.1|    3688|       Bot|


In [ ]:
WebLogsFile.write.option("header",True).csv("/content/drive/MyDrive/WebLogCsv2")

In [ ]:
# importing panda library
import pandas as pd
  
# readinag given csv file
# and creating dataframe
dataframe1 = pd.read_csv("/content/access.log",sep=" ")
  
# storing this dataframe in a csv file
dataframe1.to_csv('access.csv', index = None)

In [ ]:
WebLogsFile.write.format("csv").save("/content/drive/MyDrive/WebLogCsv")

In [ ]:
# from collections import namedtuple
# import pandas as pd
# import datetime
# import ipaddress

# #Columns of the dataset
# fields=('Host', 'datetime', 'rescode', 'size', 'method','request','protocol','user_agent')

# #namedtuple is a function that creates class called webl
# WebL= namedtuple('WebLogs', fields)


# #Parse the row list and return Movie object
# def parse(row):
#     row[0] = ipaddress.ip_address(row[0])
#     row[1] =  pd.to_datetime(row[1], format='%d/%b/%Y:%H:%M:%S %z')
#     row[2] = int(row[2])
#     row[3] = int(row[3])
#     row[4] = str(row[4])
#     row[5] = urllib.parse.unquote_plus(row[5])
#     row[6] = str(row[6])
#     row[7] = str(row[7])
#     return WebL(*row[:7])

In [ ]:
# WebLogsFinal=dataframe2.map(parse)

In [ ]:
# WebLogsFinal.take(1)